In [1]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("yelp_academic") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [59]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [22]:
from pymongo import MongoClient

mc = MongoClient()
db  = mc['raw_restaurants']
bisn = db['restaurants']
users = db['users']
rv = db['reviews']
rv_s = db['reviews_scrap']

In [23]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')

In [3]:
rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

In [5]:
user = spark.read.json('yelp_dataset/yelp_academic_dataset_user.json')

<i> Next cell is not need for now </i>

In [24]:
biz.createOrReplaceTempView('business')
rev.createOrReplaceTempView('reviews')

In [27]:
biz.count(), rev.count()

(188593, 5996996)

In [40]:
cities = biz.select('city')
list_of_c = cities.groupBy('city').count().collect()

In [42]:
len(list_of_c)

1111

In [57]:
states = biz.select('state').groupBy('state').count().collect()

In [58]:
biz.select('categories').take(2)

[Row(categories='Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel'),
 Row(categories='Chicken Wings, Burgers, Caterers, Street Vendors, Barbeque, Food Trucks, Food, Restaurants, Event Planning & Services')]

In [75]:
from train_model import str_to_l, if_rest_udf

In [62]:
str_to_l('Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel')

['Tours', 'Breweries', 'Pizza', 'Restaurants', 'Food', 'Hotels & Travel']

In [69]:
str_to_list_udf = udf(str_to_l, ArrayType(StringType()))

In [77]:
rests = biz.filter(if_rest_udf(biz.categories))